In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

In [3]:
df = pd.read_csv("02_Test_Data_Set/card_cust.csv")

In [4]:
df.head(2)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0


### 전처리

In [5]:
df[["CREDIT_LIMIT", "MINIMUM_PAYMENTS"]].isna().sum()

CREDIT_LIMIT         0
MINIMUM_PAYMENTS    74
dtype: int64

In [6]:
df["MINIMUM_PAYMENTS"] = df["MINIMUM_PAYMENTS"].fillna(df["MINIMUM_PAYMENTS"].mean())

In [7]:
base = df.copy() # 꼭 이렇게 코드를 작성할 필요는 없음.

### Q1.

In [12]:
df_q1 = base[["TENURE", "BALANCE", "CREDIT_LIMIT"]]

In [13]:
df_q1["TENURE"].unique()

array([12.,  8., 11.,  9., 10.,  7.,  6.])

In [18]:
n_t = 12

df_q1_sub = df_q1.loc[df_q1["TENURE"] == n_t, ["BALANCE", "CREDIT_LIMIT"]]
df_q1_sub.corr().iloc[0, 1]

0.4608334883447319

In [19]:
ls_corr = []
for n_t in df_q1["TENURE"].unique():
    df_q1_sub = df_q1.loc[df_q1["TENURE"] == n_t, ["BALANCE", "CREDIT_LIMIT"]]
    val_corr = df_q1_sub.corr().iloc[0, 1]
    ls_corr = ls_corr + [val_corr]

In [24]:
round(max(ls_corr), 2)

0.95

### Q2.

In [26]:
df_q2 = base.drop("CUST_ID", axis = 1)
df_q2.head(2)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0


In [27]:
df_q2_nor = StandardScaler().fit_transform(df_q2)
df_q2_nor = pd.DataFrame(df_q2_nor, columns = df_q2.columns)
df_q2_nor.head(2)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,-0.848768,-0.419879,-0.441936,-0.374048,-0.395301,-0.482354,-0.872701,-0.804321,-0.719620,-0.684701,-0.457918,-0.564116,-1.161669,-0.557396,-0.443725,-0.465544,0.282429
1,0.282791,0.012131,-0.469017,-0.374048,-0.470304,1.878468,-1.282558,-0.804321,-0.924403,0.513493,0.065417,-0.628057,0.150025,0.360574,-0.086159,0.331592,0.282429


In [31]:
k = 2

model_kmeans = KMeans(n_clusters = k, random_state = 1234)
model_kmeans.fit(df_q2_nor)
val_score = silhouette_score(df_q2_nor, model_kmeans.labels_)
val_score

0.3075281530456079

In [35]:
list(range(2, 6))

[2, 3, 4, 5]

In [32]:
ls_score = []

for k in range(2, 6):
    model_kmeans = KMeans(n_clusters = k, random_state = 1234)
    model_kmeans.fit(df_q2_nor)
    val_score = silhouette_score(df_q2_nor, model_kmeans.labels_)
    ls_score = ls_score + [val_score]

In [34]:
ls_score # k = 2 일 때 가장 좋음.

[0.3075281530456079,
 0.19636128772937608,
 0.207150984946399,
 0.19274056144483248]

In [36]:
model_kmeans = KMeans(n_clusters = 2, random_state = 1234)
model_kmeans.fit(df_q2_nor)

KMeans(n_clusters=2, random_state=1234)

In [37]:
df_q2.head(2)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0


In [52]:
model_kmeans.labels_

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,

In [38]:
df_q2["cluster"] = model_kmeans.labels_

In [53]:
df_q2.groupby("cluster")["ONEOFF_PURCHASES"].mean().round(2)

cluster
0     340.23
1    3946.19
Name: ONEOFF_PURCHASES, dtype: float64

In [40]:
df_q2.groupby("cluster")["ONEOFF_PURCHASES"].mean().round(2)[1]

3946.19

### Q3.

In [41]:
7 % 4

3

In [44]:
df_train = base.loc[base["CUST_ID"] % 4 != 0, ].drop("CUST_ID", axis = 1)
df_test  = base.loc[base["CUST_ID"] % 4 == 0, ].drop("CUST_ID", axis = 1)

In [45]:
len(df_train), len(df_test)

(752, 248)

In [46]:
df_train.head(2)

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,40.900749,0.818182,95.4,0.0,95.4,0.000000,0.166667,0.0,0.083333,0.00,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0


In [48]:
model_tree = DecisionTreeRegressor(random_state = 1234)
model_tree.fit(X = df_train.drop("ONEOFF_PURCHASES", axis = 1),
               y = df_train["ONEOFF_PURCHASES"])

DecisionTreeRegressor(random_state=1234)

In [49]:
pred = model_tree.predict(df_test.drop("ONEOFF_PURCHASES", axis = 1))
pred[:5]

array([1500.,    0., 1490.,    0.,    0.])

In [50]:
from sklearn.metrics import mean_squared_error

In [51]:
round(mean_squared_error(y_true = df_test["ONEOFF_PURCHASES"],
                         y_pred = pred) ** 0.5, 1)

1039.2